In [3]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

In [4]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16,9]
plt.style.use('ggplot')
import os
import pandas as pd
import numpy as np 
import pickle as pk
import glob
from fbprophet import Prophet
import sys
sys.path.append('../')
import wiki
from wiki import utils 
import multiprocessing as mp
total_proc = None
from tqdm import tqdm
from IPython.display import clear_output

In [5]:
PROPHET_PATH = '../data/prophet/'
RESULTS_PATH = 'results/'
VERSION = 'v7/'

In [15]:
pagedf = pd.read_feather(PROPHET_PATH+'pagedf.f')

In [16]:
pagedf.shape

(793, 145063)

In [22]:
forecast = pd.read_feather(PROPHET_PATH+VERSION+'100009df.f')

In [13]:
len(forecast[forecast.train == 1]) + len(forecast[forecast.train == 0])

733

In [17]:
len(forecast[forecast.train == 1])

613

In [14]:
len(forecast[forecast.train == 0])

120

In [20]:
VERSION2 = 'v6/'

In [28]:
forecastv6 = pd.read_feather(PROPHET_PATH+VERSION2+'100009df.f')

In [29]:
len(forecastv6)

763

In [30]:
len(forecastv6[forecastv6.train == 0])

60

In [31]:
VERSION2t = 'v6t/'

In [32]:
forecastv6t = pd.read_feather(PROPHET_PATH+VERSION2t+'100009df.f')

In [33]:
len(forecastv6t)

793

In [36]:
len(forecastv6t[forecastv6t.train == 0])

30

In [35]:
forecastv6t

,ds,t,trend,seasonal_lower,seasonal_upper,trend_lower,trend_upper,yhat_lower,yhat_upper,weekly,...,weekly_upper,yearly,yearly_lower,yearly_upper,seasonal,yhat,yhat_org,y,y_org,train
0,2015-07-01,0.000000,318.535600,-319.448984,-319.448984,318.535600,318.535600,-3482.266648,3133.068387,-99.948580,...,-99.948580,-219.500405,-219.500405,-219.500405,-319.448984,0,-0.913384,134.0,134.0,1.0
1,2015-07-02,0.001312,321.109773,-323.787847,-323.787847,321.109773,321.109773,-3436.426671,3429.404728,-120.424570,...,-120.424570,-203.363278,-203.363278,-203.363278,-323.787847,0,-2.678075,153.0,153.0,1.0
2,2015-07-03,0.002625,323.683945,-93.518419,-93.518419,323.683945,323.683945,-3025.042095,3337.705827,94.479603,...,94.479603,-187.998022,-187.998022,-187.998022,-93.518419,230,230.165526,124.0,124.0,1.0
3,2015-07-04,0.003937,326.258118,-329.976438,-329.976438,326.258118,326.258118,-3332.944680,3110.564325,-156.180599,...,-156.180599,-173.795839,-173.795839,-173.795839,-329.976438,0,-3.718320,101.0,101.0,1.0
4,2015-07-05,0.005249,328.832291,-338.667372,-338.667372,328.832291,328.832291,-3076.751962,3262.124232,-177.569733,...,-177.569733,-161.097639,-161.097639,-161.097639,-338.667372,0,-9.835081,138.0,138.0,1.0
5,2015-07-06,0.006562,331.406464,323.534641,323.534641,331.406464,331.406464,-2359.539799,4091.853340,473.718326,...,473.718326,-150.183685,-150.183685,-150.183685,323.534641,655,654.941104,172.0,172.0,1.0
6,2015-07-07,0.007874,333.980636,-155.339869,-155.339869,333.980636,333.980636,-3122.004558,3684.100310,-14.074447,...,-14.074447,-141.265423,-141.265423,-141.265423,-155.339869,179,178.640767,130.0,130.0,1.0
7,2015-07-08,0.009186,336.554809,-234.428330,-234.428330,336.554809,336.554809,-3447.743994,3168.755941,-99.948580,...,-99.948580,-134.479751,-134.479751,-134.479751,-234.428330,102,102.126479,167.0,167.0,1.0
8,2015-07-09,0.010499,339.128982,-250.310475,-250.310475,339.128982,339.128982,-3243.853179,3275.672499,-120.424570,...,-120.424570,-129.885905,-129.885905,-129.885905,-250.310475,89,88.818507,150.0,150.0,1.0
9,2015-07-10,0.011811,341.703154,-32.985440,-32.985440,341.703154,341.703154,-2857.667178,3652.206204,94.479603,...,94.479603,-127.465043,-127.465043,-127.465043,-32.985440,309,308.717714,166.0,166.0,1.0
